# Plot Project Car Data in Real Time

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#from types import SimpleNamespace 
#import time

PORT = "/dev/cu.usbmodemM43210051"
BAUD = 9600

### Read Data

Use the pySerial library to read data from USB serial and save it as an array.
The data is in the form of a JSON object with the sensor outputs

In [ ]:
import serial
import json

class ReadChar:
    def __init__(self, serial):
        self.buff = []
        self.serial = serial

    def updateBuff(self):
        for i in range(100):
            b = self.serial.read()
            while (True):
                try:
                    self.buff.append(b.decode('utf-8'))
                    break
                except(UnicodeDecodeError):
                    b = self.serial.read()
                    continue

    def readChar(self):
        if len(self.buff) == 0:
            self.updateBuff()
        char = self.buff[0]
        self.buff = self.buff[1:]
        return char

def getData(s):
    readChar = ReadChar(s)

    dataRead = ""
    char = readChar.readChar()

    #keep reading until start marker is reached
    while char != 'S':
        char = readChar.readChar()

    #read everything between start and end markers
    if char == 'S':
        while char != '{':
            char = readChar.readChar()
        while char != 'E':
            dataRead += char
            char = readChar.readChar()
    
    data = json.loads(dataRead)
    sensorData = np.array(data["sensor"])
    return sensorData

### Plot Data

Use matplotlib matshow to plot the array as a heatmap

In [ ]:
from IPython import display

while(True):

    data = getData(PORT, BAUD)
    display.clear_output(wait=True)
    plt.figure()
    plt.title("IR Sensor Output")
    data = np.expand_dims(data, axis=0)
    color_map = plt.imshow(data)
    color_map.set_cmap("Blues_r")
    plt.colorbar()
    plt.show()